In [2]:
import math
from solid import * # pip install solidpython

def make_sphere_at_pos(size_d, xyz): # 直径, 場所[x,y,z]
    return translate(xyz)(sphere(d=size_d, segments=4)) # 4は形状の滑らかさ

def make_cube_at_pos(size_xyz, xyz):
    return translate(xyz)(cube(size_xyz, center=True))

x_num = 10#30 # 60 for 1.8mm, 40 for 3mm
y_num = 10#30 # 60
r = 1.5 # 0.9 # 1.5 # 球の半径（mm）
d = 2 * r # 球の直径でもあり、広い方向のspaceでもある
narrow_space=math.sqrt(3)*r # 狭い方向のspace
base_thickness = 4  # 厚み(mm)
base_height = 5     # 高さ(mm)
base_over_height=10 # 余分な高さ（mm）
edge_length = 3     # 周囲幅(mm)
base_x_size = (x_num) * narrow_space+2*edge_length
base_y_size = (x_num) * d + 2*edge_length
base_x_outer_size = base_x_size + 2*base_thickness
base_y_outer_size = base_y_size + 2*base_thickness
silicone_thickness = 4 # mm

# 直方体を作る
mold_base = make_cube_at_pos([base_x_outer_size, 
                              base_y_outer_size,
                              base_height+base_thickness+base_over_height],
                             [base_x_size/2, 
                              base_y_size/2, 
                              (base_height+base_thickness+base_over_height)/2])
# 直方体を削る
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              base_y_size,
                              base_over_height+base_thickness],
                             [base_x_size/2,
                              base_y_size/2, 
                              base_height+base_thickness+(base_over_height+base_thickness)/2])
# 周囲の「お堀」部分
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              silicone_thickness,
                              base_height+base_over_height+base_thickness],
                             [base_x_size/2,
                              silicone_thickness/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [silicone_thickness, 
                              base_y_size,
                              base_height+base_over_height+base_thickness],
                             [silicone_thickness/2,
                              base_y_size/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [base_x_size, 
                              silicone_thickness,
                              base_height+base_over_height+base_thickness],
                             [base_x_size/2,
                              base_y_size-silicone_thickness/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])
mold_base = mold_base - make_cube_at_pos(
                             [silicone_thickness, 
                              base_y_size,
                              base_height+base_over_height+base_thickness],
                             [base_x_size-silicone_thickness/2,
                              base_y_size/2, 
                              base_thickness+(base_height+base_over_height+base_thickness)/2])

# 球形部分を除去（3Dプリンタだけで型を作るなら足すところだが、ビーズ球を使うため、抜く）
spheres = []
for y in range(y_num):
    Y=y*d+r+edge_length
    Z=base_height+base_thickness 
    for x in range(x_num):
        X=x*narrow_space+r+edge_length
        if x%2 == 0:
            spheres.append( make_sphere_at_pos(d,[X,Y,Z])   )
        else:
            spheres.append( make_sphere_at_pos(d,[X,Y+r,Z]) )
# 最終的に生成された形状
mold_base = difference()(mold_base,spheres)
# .scad ファイルに出力する
scad_render_to_file(mold_base, "lensMold.scad")

In [3]:
import subprocess

# OpenSCADを使って、.scad ファイルをSTLファイルに変換する
openscad = '/Applications/OpenSCAD.app/Contents/MacOS/OpenSCAD'
subprocess.run([openscad, 'lensMold.scad','-o lensMold.stl','-q'])

'/Users/jun/source/SD/SD202312_hologram/lensMold.scad'